This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, your task is to predict whether a given blight ticket will be paid on time.

All data through the [Detroit Open Data Portal](https://data.detroitmi.gov/). Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this is the Area Under the ROC Curve (AUC). 
___

Create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Rules

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score, recall_score
import pandas as pd
import numpy as np
def blight_model():
    df_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    df_test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    add = pd.read_csv('addresses.csv')
    latlon = pd.read_csv('latlons.csv')
    df_train = (latlon.set_index('address').join(add.set_index('address')).set_index('ticket_id')).join(df_train.set_index('ticket_id'),how = 'right')
    df_test = (latlon.set_index('address').join(add.set_index('address')).set_index('ticket_id')).join(df_test.set_index('ticket_id'), how = 'right')
    df_train = df_train[np.isfinite(df_train['compliance'])]
    features_to_drop_train = ['payment_amount','payment_date', 'payment_status','balance_due','collection_status','compliance_detail']
    df_train.drop(features_to_drop_train, axis = 1,inplace = True)
    import datetime
    def t_gap(hearing_date, ticket_issue_date):
        if not hearing_date or type(hearing_date)!=str:return 73
        ticket_issue = datetime.datetime.strptime(ticket_issue_date, '%Y-%m-%d %H:%M:%S')
        hearing_date_1 = datetime.datetime.strptime(hearing_date, '%Y-%m-%d %H:%M:%S')
        gap = hearing_date_1 - ticket_issue
        return gap.days
    df_train['days'] = df_train.apply(lambda x:t_gap(x['hearing_date'], x['ticket_issued_date']), axis = 1)
    df_test['days'] = df_test.apply(lambda x:t_gap(x['hearing_date'], x['ticket_issued_date']), axis = 1)
    features_to_bin = ['agency_name', 'state', 'disposition']
    df_train = pd.get_dummies(df_train, columns = features_to_bin)
    df_test = pd.get_dummies(df_test, columns = features_to_bin) 
    df_train.drop(df_train.columns[3:25].tolist(), axis = 1,inplace = True)
    df_test.drop(df_test.columns[2:25].tolist(), axis = 1, inplace = True)
    df_train_x = df_train.drop(['compliance'], axis = 1)
    df_train_y = df_train['compliance']
    df_train_x.drop(['inspector_name'], axis = 1, inplace = True)
    df_train_x['lat'].fillna(method = 'pad', inplace = True)
    df_train_x['lon'].fillna(method = 'pad', inplace = True)
    df_train_x1 = df_train[['disposition_Responsible (Fine Waived) by Deter','disposition_Responsible by Admission',
                       'disposition_Responsible by Default', 'disposition_Responsible by Determination']]
    df_test['lat'].fillna(method = 'pad', inplace = True)
    df_test['lon'].fillna(method = 'pad', inplace = True)
    for i in df_train_x.columns:
        for j in set(df_train_x.columns.tolist()) ^ set(df_test.columns.tolist()):
            if i == j:
                df_train_x.drop([i], axis = 1, inplace = True)
    for i in df_test.columns:
        for j in set(df_train_x.columns.tolist()) ^ set(df_test.columns.tolist()):
            if i == j:
                df_test.drop([i], axis = 1, inplace = True)
    
    clf = RandomForestClassifier(n_estimators = 25, max_depth = 15, max_features = None, random_state = 3)

    clf1 = clf.fit(df_train_x, df_train_y)
    y_pred = clf1.predict_proba(df_test)
    df_test['compliance'] = y_pred[:,1]

    return df_test['compliance']
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.040418
285362    0.048762
285361    0.048762
285338    0.032824
285346    0.033477
285345    0.033477
285347    0.032824
285342    0.711471
285530    0.043415
284989    0.042331
285344    0.037751
285343    0.037751
285340    0.039256
285341    0.039256
285349    0.047426
285348    0.047426
284991    0.064768
285532    0.047073
285406    0.043309
285001    0.111181
285006    0.111181
285405    0.050482
285337    0.033169
285496    0.053822
285497    0.053822
285378    0.049951
285589    0.048766
285585    0.049662
285501    0.053822
285581    0.049810
            ...   
376367    0.031678
376366    0.031678
376362    0.339938
376363    0.339938
376365    0.031678
376364    0.031678
376228    0.169545
376265    0.040388
376286    0.070046
376320    0.033126
376314    0.018652
376327    0.232573
376385    0.312529
376435    0.324467
376370    0.324467
376434    0.063302
376459    0.065554
376478    0.065554
376473    0.032675
376484    0.041028
376482    0.194047
37